In [4]:
!pip install pandas
!pip install numpy

In [5]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

In [8]:
problem = pd.read_csv('problem.csv')
user = pd.read_csv('user.csv')
target_problem = "1000"

In [9]:
print(problem)

       문제번호                                    링크           난이도       제목  \
0      1000  https://www.acmicpc.net/problem/1000      Bronze V      A+B   
1      1001  https://www.acmicpc.net/problem/1001      Bronze V      A-B   
2      1007  https://www.acmicpc.net/problem/1007       Gold II    벡터 매칭   
3      1008  https://www.acmicpc.net/problem/1008     Bronze IV      A/B   
4      1009  https://www.acmicpc.net/problem/1009    Bronze III     분산처리   
...     ...                                   ...           ...      ...   
18197  2549  https://www.acmicpc.net/problem/2549        Gold I  루빅의 사각형   
18198  2794  https://www.acmicpc.net/problem/2794    Platinum I       화성   
18199  3110  https://www.acmicpc.net/problem/3110      Gold III      부등식   
18200  3148  https://www.acmicpc.net/problem/3148  Platinum III       빙고   
18201  4160  https://www.acmicpc.net/problem/4160    Platinum V       이혼   

             분류     해결  평균시도  
0            수학  98776  2.26  
1            수학  81082  1

In [10]:
problem_df = problem[['문제번호','난이도','분류','해결','평균시도']]
print(problem_df)

       문제번호           난이도        분류     해결  평균시도
0      1000      Bronze V        수학  98776  2.26
1      1001      Bronze V        수학  81082  1.39
2      1007       Gold II        수학    858  2.72
3      1008     Bronze IV        수학  53039  3.01
4      1009    Bronze III        수학   6579  3.82
...     ...           ...       ...    ...   ...
18197  2549        Gold I  중간에서 만나기    234  3.18
18198  2794    Platinum I  중간에서 만나기     11  3.00
18199  3110      Gold III  중간에서 만나기     38  2.79
18200  3148  Platinum III  중간에서 만나기      3  4.00
18201  4160    Platinum V  중간에서 만나기      9  2.67

[18202 rows x 5 columns]


In [11]:
pd.set_option('max_colwidth',100)
#각각의 장르와 그 장르의 id를 추출해 낼 수 있다
problem_df[['분류','난이도']][:1]

,분류,난이도
0,수학,Bronze V


In [28]:
#literal_eval() 함수를 사용하면 이 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다
from ast import literal_eval
problem_df['분류'] = problem_df['분류'].apply(literal_eval) #genres 칼럼은 문자열이 아닌 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체들을 가진다
problem_df['난이도'] = problem_df['난이도'].apply(literal_eval)


ValueError: malformed node or string: <_ast.Name object at 0x000001FC8452A160>

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
count_vect  = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(problem_df['분류'])
print(genre_mat.shape)

(18202, 151)


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim)

(18202, 18202)
[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]]


In [16]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])


[[    0  1142  1140 ... 12126 12125  9100]]


In [17]:
#장르 유사도에 따라 영화를 추천하는 함수
#함수명 : find_sim_movie()
#movies_df DataFrame : 기반 데이터
#genre_sim_sorted_ind : 레코드별 장르 코사인 유사도 인덱스
#DataFrame : 고객이 선정한 추천 기준이 되는 영화제목, 추천할 영화 건수를 입력하면 추천 영화 정보를 가짐
def find_sim_problem(df, sorted_ind, title_name, top_n = 10):
    
    title_problem = df[df['문제번호']== int(title_name)]
    
    title_index = title_problem.index.values
    similar_indexes = sorted_ind[title_index,:(top_n)]
    
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [99]:
similar_movies = find_sim_problem(problem_df, genre_sim_sorted_ind,target_problem, 40)
similar_movies[['문제번호','평균시도','해결','난이도','분류']]

,문제번호,평균시도,해결,난이도,분류
0,1000,2.26,98776,Bronze V,수학
1142,1193,1.88,14902,Bronze II,수학
1140,1187,2.32,34,Diamond V,수학
1139,1183,2.21,43,Silver V,수학
1138,1179,2.84,85,Gold III,수학
...,...,...,...,...,...
12584,5533,1.39,1263,Bronze I,사칙연산
12583,5532,1.56,3586,Bronze IV,사칙연산
12582,5522,1.12,2621,Bronze V,사칙연산
12581,5361,1.52,1583,Bronze III,사칙연산


In [18]:
problem_df[['문제번호','난이도','평균시도']].sort_values('평균시도',ascending=True)[:10]



,문제번호,난이도,평균시도
15157,16371,Diamond V,1.0
17750,18350,Diamond I,1.0
12830,3858,Platinum I,1.0
17748,18163,Ruby II,1.0
17747,17879,Platinum II,1.0
16392,19327,Diamond IV,1.0
13547,6752,Silver V,1.0
12832,5041,Platinum II,1.0
16109,2214,Gold III,1.0
14659,11373,Platinum III,1.0


In [59]:
#난이도를 정제하여 난이도 수준을 맞춤
level_array = ['Bronze','Silver','Gold','Platium','Diamond','Ruby']
print(level_array[0]) 
def level_set(df,target_problem):
    level_idx = df.index[df['문제번호']==int(target_problem)]
    level = df.loc[level_idx[0]]['난이도'].split(' ')[0]
    df['난이도 정제'] = df['난이도'].str.split(' ')
    print(df['난이도 정제'].contains('Bronze'))
level_set(problem_df,target_problem)
print(return_df)

Bronze


AttributeError: 'Series' object has no attribute 'contains'

In [20]:
C = problem_df['평균시도'].mean()
m = problem_df['해결'].quantile(0.6)
print('C: ',round(C,3),'m: ',round(m,3))

C:  3.205 m:  229.0


In [24]:
#원래 점수에 
percentile = 0.6
m = problem_df['해결'].quantile(0.6)
C = problem_df['평균시도'].mean()

def weighted_vote_average(record):
    v = record['해결']
    R = record['평균시도']

    return  (v/(v+m)) * R + (v/(v+m)) * C


problem['weighted_vote'] = problem_df.apply(weighted_vote_average,axis=1)
print(problem.head(3))

   문제번호                                    링크       난이도     제목  분류     해결  \
0  1000  https://www.acmicpc.net/problem/1000  Bronze V    A+B  수학  98776   
1  1001  https://www.acmicpc.net/problem/1001  Bronze V    A-B  수학  81082   
2  1007  https://www.acmicpc.net/problem/1007   Gold II  벡터 매칭  수학    858   

   평균시도  weighted_vote  
0  2.26       5.451965  
1  1.39       4.581664  
2  2.72       4.676459  


In [41]:
problem = problem.drop_duplicates(['문제번호'],keep='last')
#새로 부여된 weighted_vote 평점이 높은 순으로 상위 10개 추출
problem[['문제번호','제목','난이도','평균시도','weighted_vote','해결']].sort_values(
    'weighted_vote',ascending=False)[:10] 

,문제번호,제목,난이도,평균시도,weighted_vote,해결
17227,13705,Ax+Bsin(x)=C,Diamond IV,31.84,12.180746,122
14910,2261,가장 가까운 두 점,Platinum IV,6.59,8.598357,1646
12318,1600,말이 되고픈 원숭이,Gold V,5.70,8.178668,2580
11500,1002,터렛,Silver IV,4.98,8.043032,13010
10518,1654,랜선 자르기,Silver III,5.06,7.990116,6666
17524,1016,제곱 ㄴㄴ 수,Gold I,5.17,7.764040,2912
10594,3649,로봇 프로젝트,Gold IV,6.59,7.753691,870
17767,1005,ACM Craft,Gold III,4.88,7.675913,4301
13310,1325,효율적인 해킹,Silver II,4.80,7.381752,2714
9020,1107,리모컨,Gold V,4.49,7.379161,5357


In [53]:
#이제 새롭게 정의된 평점 기준에 따라서 영화를 추천한다
#장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정한 뒤에 weighted_vote 칼럼이 높은 순으로 top_n만큼 추출하는 방식으로
#find_sim_movie()함수를 변경한다

def find_sim_movie(df, sorted_ind, title_name, top_n =3):
    title_problem = df[df['문제번호'] == int(title_name)]
    title_index = title_problem.index.values
    print(df)
    #top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexs = similar_indexes.reshape(-1)
    print(df.shape)
    print(similar_indexes)
    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes!=title_index]
    print(similar_indexes)
    #top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[df['문제번호']== (similar_indexes)].sort_values('weighted_vote',ascending=False)[:top_n]

similar_problem = find_sim_movie(problem,genre_sim_sorted_ind,target_problem,10)
similar_problem[['문제번호','난이도','weighted_vote']]

       문제번호                                    링크           난이도  \
1605   1011  https://www.acmicpc.net/problem/1011      Silver I   
1608   1019  https://www.acmicpc.net/problem/1019        Gold I   
1609   1024  https://www.acmicpc.net/problem/1024    Silver III   
1614   1049  https://www.acmicpc.net/problem/1049     Silver IV   
1616   1059  https://www.acmicpc.net/problem/1059      Silver V   
...     ...                                   ...           ...   
18197  2549  https://www.acmicpc.net/problem/2549        Gold I   
18198  2794  https://www.acmicpc.net/problem/2794    Platinum I   
18199  3110  https://www.acmicpc.net/problem/3110      Gold III   
18200  3148  https://www.acmicpc.net/problem/3148  Platinum III   
18201  4160  https://www.acmicpc.net/problem/4160    Platinum V   

                                 제목        분류    해결  평균시도  weighted_vote  
1605   Fly me to the Alpha Centauri        수학  8980  3.50       6.537881  
1608                          책 페이지        수학

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
ç